In [1]:
import sys
sys.path.insert(0, '/Users/ahmedbegga/Desktop/TFG-Ahmed/SetXAI/')

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from src.model import *
from data.MnistSet import MNISTSet
from data.MnistSet import get_loader
import trimesh
from matplotlib import pyplot as plt
import numpy as np
import h5py
from torch.utils.data import DataLoader, Dataset, TensorDataset
from time import sleep
from tqdm import tqdm
from data.pointcloud_utils import *
from data.Modelnet10toSet import *

In [15]:
import glob
import os
import os.path as osp
import shutil

import torch

from torch_geometric.data import InMemoryDataset, download_url, extract_zip
from torch_geometric.io import read_off
from torch_geometric.datasets import ModelNet
dataset = ModelNet(root='./',name='10').shuffle()

Extracting ./ModelNet10.zip
Processing...
Done!


In [25]:
dataset[900]

Data(pos=[1447, 3], face=[3, 2544], y=[1])

In [ ]:
DATA_DIR = "/Users/ahmedbegga/Desktop/TFG-Ahmed/SetXAI/DeepSets Clasification PointCloud/10/raw"

In [ ]:
mesh = trimesh.load(os.path.join(DATA_DIR, "chair/train/chair_0001.off"))
points = mesh.sample(10000)
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(111, projection="3d")
ax.scatter(points[:, 0], points[:, 1], points[:, 2],color = 'r')
ax.set_axis_off()
plt.show()


In [ ]:
def parse_dataset(num_points=2048):

    train_points = []
    train_labels = []
    test_points = []
    test_labels = []
    class_map = {}
    folders = glob.glob(os.path.join(DATA_DIR, "[!README]*"))

    for i, folder in enumerate(folders):
        print("processing class: {}".format(os.path.basename(folder)))
        # store folder name with ID so we can retrieve later
        class_map[i] = folder.split("/")[-1]
        # gather all files
        train_files = glob.glob(os.path.join(folder, "train/*"))
        test_files = glob.glob(os.path.join(folder, "test/*"))

        for f in train_files:
            train_points.append(trimesh.load(f).sample(num_points))
            train_labels.append(i)

        for f in test_files:
            test_points.append(trimesh.load(f).sample(num_points))
            test_labels.append(i)

    return (
        np.array(train_points),
        np.array(test_points),
        np.array(train_labels),
        np.array(test_labels),
        class_map,
    )

NUM_POINTS = 2048
NUM_CLASSES = 10
BATCH_SIZE = 32

train_points, test_points, train_labels, test_labels, CLASS_MAP = parse_dataset(
    NUM_POINTS
)

In [ ]:
train_points[0]

In [ ]:
h5f = h5py.File('pointcloud.h5', 'w')
h5f.create_dataset('train_points', data=train_points)
h5f.create_dataset('test_points', data=test_points)
h5f.create_dataset('train_labels', data=train_labels)
h5f.create_dataset('test_labels', data=test_labels)
h5f.close()

In [ ]:
h5f.close()

In [ ]:
h5f = h5py.File('pointcloud.h5','r')
train_points = h5f['train_points'][:]
test_points = h5f['test_points'][:]
train_labels = h5f['train_labels'][:]
test_labels = h5f['test_labels'][:]

In [ ]:
train_points[0]

In [ ]:
#train_points = train_points.reshape(train_points.shape[0],3,2048)
#test_points = test_points.reshape(test_points.shape[0],3,2048)

In [ ]:
train_points.shape

In [ ]:
train_labels[800]

In [ ]:
# Les aplicamos un poco de ruido a las figuras y les metemos un suffle
def augment(points):
    # jitter points
    points += np.random.uniform(points.shape, -0.005, 0.005, dtype=tf.float64)
    # shuffle points
    points = np.random.shuffle(points)
    return points
torch_train = TensorDataset(torch.from_numpy(train_points).float(), torch.from_numpy(train_labels).long())
torch_test = TensorDataset(torch.from_numpy(test_points).float(), torch.from_numpy(test_labels).long())

    

In [ ]:
train_loader = DataLoader(torch_train, batch_size=32)
test_loader = DataLoader(torch_test, batch_size=32)

In [ ]:
data_iter = iter(train_loader)
p,l=data_iter.next()

In [ ]:
p,l=data_iter.next()
l

In [ ]:
a =p.squeeze(0).transpose(1,2).numpy()
print(a.shape)

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(111, projection="3d")
ax.scatter(a[0,0,:], a[0,1,:], a[0,2, :],color = 'r')
ax.set_axis_off()
plt.show()

In [3]:
train_data = PointCloudData()
test_data = PointCloudData(None, Train=False, folder='test', transform=None)
train_loader = DataLoader(train_data, batch_size=32)
test_loader = DataLoader(test_data, batch_size=32)

In [4]:
set_channels = 3
set_size = 512
  
hidden_dim = 64
iters = 10
latent_dim = 32
lr = 0.001
n_epochs = 100
net = MaxEncoder(set_channels,latent_dim,hidden_dim)

In [5]:
net

MaxEncoder(
  (conv): Sequential(
    (0): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
    (1): ReLU(inplace=True)
    (2): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
  )
  (lin): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=64, out_features=32, bias=True)
  )
  (classifier): Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=10, bias=True)
  )
)

In [6]:
optimizer = torch.optim.Adam([p for p in net.parameters() if p.requires_grad], lr=0.001)

In [7]:
net.train()
mask = torch.ones(512).float()
for epoch in range(n_epochs):
    with tqdm(train_loader, unit="batch") as tepoch:
        for i, sample in enumerate(tepoch):
            tepoch.set_description(f"Epoch {epoch}")
            label,points = map(lambda x: x, sample)
            #print(points.shape)
            
            optimizer.zero_grad()
            output = net(points,mask)
            loss = F.cross_entropy(output, label)
            acc = (output.max(dim=1)[1] == label).float().mean()
            #print(output.max(dim=1))
            loss.backward()
            optimizer.step()
            tepoch.set_postfix(loss=loss.item(), acc=100. * acc.item())

Epoch 0:   4%|██▍                                                          | 5/125 [00:20<08:20,  4.17s/batch, acc=0, loss=2.4]


KeyboardInterrupt: 